In [1]:
import quandl  # Access to Sharadar Core US Equities Bundle
api_key = '7B87ndLPJbCDzpNHosH3'

import math
import platform
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np
import torch
import pandas as pd
from IPython.display import display
import time

from datetime import date, datetime, time, timedelta


print("Python version: ", platform.python_version())
print("Pytorch version: {}".format(torch.__version__))

Python version:  3.6.9
Pytorch version: 1.2.0


### Import Master Label Set

In [2]:
stocks = pd.read_csv("stock_tickers_sorted.cvs")
stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10207 entries, 0 to 10206
Data columns (total 30 columns):
ticker            10207 non-null object
appreciation      10207 non-null float64
10bagger          10207 non-null bool
table             10207 non-null object
permaticker       10207 non-null int64
name              10207 non-null object
exchange          10207 non-null object
isdelisted        10207 non-null object
category          10207 non-null object
cusips            10202 non-null object
siccode           9865 non-null float64
sicsector         9865 non-null object
sicindustry       9865 non-null object
famasector        0 non-null float64
famaindustry      9621 non-null object
sector            9727 non-null object
industry          10207 non-null object
scalemarketcap    8809 non-null object
scalerevenue      8441 non-null object
relatedtickers    3907 non-null object
currency          10207 non-null object
location          10165 non-null object
lastupdated       10207

In [3]:
stocks = stocks.set_index('ticker')  # set ticker as index

stocks

,appreciation,10bagger,table,permaticker,name,exchange,isdelisted,category,cusips,siccode,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
ticker,,,,,,,,,,,,,,,,,,,,,
A,6.339117,False,SEP,196290,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,3826.0,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
AA,1.224348,False,SEP,124392,Alcoa Corp,NYSE,N,Domestic,013872106,3350.0,...,USD,New York; U.S.A,2020-01-14,2016-11-01,2016-11-01,2020-01-14,2014-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
AAAGY,1.275556,False,SEP,120538,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,2834.0,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAAP,3.331837,False,SEP,155760,Advanced Accelerator Applications SA,NASDAQ,Y,ADR,00790T100,2834.0,...,USD,France,2018-06-28,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
AAC,0.099459,False,SEP,187592,AAC Holdings Inc,NYSE,Y,Domestic,000307108,8093.0,...,USD,Tennessee; U.S.A,2019-10-25,2015-09-11,2014-10-02,2019-10-25,2013-09-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYME,1.244615,False,SEP,123153,Zymeworks Inc,NYSE,N,Domestic,98985W102,2834.0,...,USD,British Columbia; Canada,2020-01-14,2017-09-02,2017-04-28,2020-01-14,2015-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zymeworks.com
ZYNE,0.333538,False,SEP,157146,Zynerba Pharmaceuticals Inc,NASDAQ,N,Domestic,98986X109,2834.0,...,USD,Pennsylvania; U.S.A,2020-01-14,2016-04-15,2015-08-05,2020-01-14,2013-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zynerba.com
ZYTO,2.727273,False,SEP,188776,Zyto Corp,OTC,Y,Domestic,989872106,3841.0,...,USD,Utah; U.S.A,2018-07-02,2016-02-17,2008-01-30,2016-01-22,2009-12-31,2012-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN


In [4]:
stocks['firstpricedate']= pd.to_datetime(stocks['firstpricedate'])
stocks['lastpricedate']= pd.to_datetime(stocks['lastpricedate'])

In [5]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10207 entries, A to ZZ
Data columns (total 29 columns):
appreciation      10207 non-null float64
10bagger          10207 non-null bool
table             10207 non-null object
permaticker       10207 non-null int64
name              10207 non-null object
exchange          10207 non-null object
isdelisted        10207 non-null object
category          10207 non-null object
cusips            10202 non-null object
siccode           9865 non-null float64
sicsector         9865 non-null object
sicindustry       9865 non-null object
famasector        0 non-null float64
famaindustry      9621 non-null object
sector            9727 non-null object
industry          10207 non-null object
scalemarketcap    8809 non-null object
scalerevenue      8441 non-null object
relatedtickers    3907 non-null object
currency          10207 non-null object
location          10165 non-null object
lastupdated       10207 non-null object
firstadded        10207 non-nul

### Generate Label Sets

In [6]:
# Extract all tickers
tickers = stocks.index.tolist()

data_list = []

# There will be 9 label sets
start_date = '03-01-2009'
end_dates = ['12-31-2010',   # 20 months
             '12-31-2011',   # 32 months
             '12-31-2012',   # 44 months
             '12-31-2013',   # 56 months
             '12-31-2014',   # 68 months
             '12-31-2015',   # 80 months
             '12-31-2016',   # 92 months
             '12-31-2017',   # 104 months
             '12-31-2018',   # 116 months
]

offset = timedelta(days=180)

for end_date in end_dates:
    data = []
    for ticker in tickers:
        
        # For a ticker to be included in a label set, it has to be active for at least 180 days
        end = datetime.strptime(end_date, '%m-%d-%Y') - offset

        if (stocks.loc[ticker]['firstpricedate'] < end):
            # print(ticker)
            # print(stocks.loc[ticker]['firstpricedate'])
            data.append(ticker)
    
    print(len(data))   # Print out the number of valid tickers in each label set     
    
    data_list.append(data)
        

6550
6860
7115
7422
7849
8703
9018
9404
9881


### Save to Label Files

In [7]:
end_dates = ['12-31-2010',   # 20 months
             '12-31-2011',   # 32 months
             '12-31-2012',   # 44 months
             '12-31-2013',   # 56 months
             '12-31-2014',   # 68 months
             '12-31-2015',   # 80 months
             '12-31-2016',   # 92 months
             '12-31-2017',   # 104 months
             '12-31-2018',   # 116 months
]

for i, end_date in enumerate(end_dates):
    print(i, end_date)
    print(len(data_list[i]))
    
    df = pd.DataFrame(data_list[i], columns =['ticker'])  # convert ticker list to dataframe
    labels = df.set_index('ticker').join(stocks)  # Join with the master label dataframe
    
    # Save to label file
    file_name = '../datasets/sharader/labels_'+end_date+'.csv'
    labels.to_csv (file_name, header=True)

0 12-31-2010
6550
1 12-31-2011
6860
2 12-31-2012
7115
3 12-31-2013
7422
4 12-31-2014
7849
5 12-31-2015
8703
6 12-31-2016
9018
7 12-31-2017
9404
8 12-31-2018
9881


## Check label files

In [8]:
labels = pd.read_csv("../datasets/sharader/labels_12-31-2018.csv")
labels['firstpricedate']= pd.to_datetime(labels['firstpricedate'])

labels = labels.sort_values(by=['ticker'])

In [9]:
labels

,ticker,appreciation,10bagger,table,permaticker,name,exchange,isdelisted,category,cusips,...,currency,location,lastupdated,firstadded,firstpricedate,lastpricedate,firstquarter,lastquarter,secfilings,companysite
0,A,6.339117,False,SEP,196290,Agilent Technologies Inc,NYSE,N,Domestic,00846U101,...,USD,California; U.S.A,2020-01-14,2014-09-26,1999-11-18,2020-01-14,1997-06-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.agilent.com
1,AA,1.224348,False,SEP,124392,Alcoa Corp,NYSE,N,Domestic,013872106,...,USD,New York; U.S.A,2020-01-14,2016-11-01,2016-11-01,2020-01-14,2014-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.alcoa.com
2,AAAGY,1.275556,False,SEP,120538,Altana Aktiengesellschaft,NYSE,Y,ADR,02143N103,...,USD,Jordan,2018-10-16,2018-02-13,2002-05-22,2010-08-12,2000-12-31,2005-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
3,AAAP,3.331837,False,SEP,155760,Advanced Accelerator Applications SA,NASDAQ,Y,ADR,00790T100,...,USD,France,2018-06-28,2016-05-19,2015-11-11,2018-02-09,2012-12-31,2016-12-31,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
4,AAC,0.099459,False,SEP,187592,AAC Holdings Inc,NYSE,Y,Domestic,000307108,...,USD,Tennessee; U.S.A,2019-10-25,2015-09-11,2014-10-02,2019-10-25,2013-09-30,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9876,ZYME,1.244615,False,SEP,123153,Zymeworks Inc,NYSE,N,Domestic,98985W102,...,USD,British Columbia; Canada,2020-01-14,2017-09-02,2017-04-28,2020-01-14,2015-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zymeworks.com
9877,ZYNE,0.333538,False,SEP,157146,Zynerba Pharmaceuticals Inc,NASDAQ,N,Domestic,98986X109,...,USD,Pennsylvania; U.S.A,2020-01-14,2016-04-15,2015-08-05,2020-01-14,2013-12-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zynerba.com
9878,ZYTO,2.727273,False,SEP,188776,Zyto Corp,OTC,Y,Domestic,989872106,...,USD,Utah; U.S.A,2018-07-02,2016-02-17,2008-01-30,2016-01-22,2009-12-31,2012-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,NaN
9879,ZYXI,3.719008,False,SEP,198130,Zynex Inc,NASDAQ,N,Domestic,98986M103,...,USD,Colorado; U.S.A,2020-01-14,2016-02-17,2004-02-25,2020-01-14,2008-03-31,2019-09-30,https://www.sec.gov/cgi-bin/browse-edgar?actio...,http://www.zynex.com


In [10]:
display(labels["ticker"].value_counts())
display(labels["10bagger"].value_counts())

KTOS    1
NHR1    1
AFG     1
FDUS    1
IMO     1
       ..
TTGT    1
GASS    1
HCCH    1
ECHO    1
TSRI    1
Name: ticker, Length: 9881, dtype: int64

False    9350
True      531
Name: 10bagger, dtype: int64